In [12]:
import requests
from requests import exceptions
from bs4 import BeautifulSoup
import urllib3
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [71]:
urllib3.disable_warnings()

In [228]:
def page_driver_this(item_id):
    
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe')
    #options.headless = True
    #driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe', options=options)  
    driver.maximize_window()
    driver.get('https://www.franke.com/ae/en/ws/products/product-search/product-explorer-results.html#akCache=true&p=5&t=grid&f=cat2_2')
    
    try:

        search_bar = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, 'pe-search__input'))
        )
        search_bar.send_keys(item_id)

        search_icon = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'pe-search__button'))
        )
        search_icon.click()

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/1.0)")

        search_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'm22-product-teaser-module__link'))
        )
        search_btn.click()

        url = driver.current_url
        return url


    except:
        pass

In [33]:
df = pd.read_excel(r'Franke_source.xlsx')

In [58]:
def getURLs():
    url_lst = []
    for i in df.index:
        item_url = page_driver_this(str(df['Item Code '][i]))
        url_lst.append(item_url)
    return url_lst

In [ ]:
urls = getURLs()
df['webpage_to_be_scraped'] = urls

In [62]:
df.to_excel('Franke_links.xlsx')

In [69]:
def create_soup(url):
    try:
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, 'lxml')
        return soup
    except:
        return 'N/A'

In [229]:
soup = create_soup('https://www.franke.com/ae/en/ws/products/dryers/hand-dryers/rodx310_detail.html')

In [132]:
def item_name(soup):
    try:
        name = soup.find('h1', {'class' : 'm19-product-picture-with-options-module-product-top-text-title'}).text.strip()
        return name
    except:
        pass

In [145]:
def item_description(soup):
    try:
        description = soup.find('div', {'class' : 'm19-product-picture-with-options-module-product-top-text'})
        description = description.find_next('div').contents[0]
        description = str(description).strip()
        return description
    except:
        pass

In [148]:
def product_number_table(soup):
    try:
        body = soup.find('div', {'class' : 'table table--grey'}).find('table').findAll('tr')
        inner_dictionary = {}

        for row in body:
            cols = row.findChildren(recursive=False)
            cols = [ele.text.strip() for ele in cols]
            inner_dictionary.update({cols[0] : cols[1]})
        return inner_dictionary
    except:
        pass

In [158]:
def product_properties_table(soup):
    try:
        body = soup.find('div', {'class' : 'table table--grey'}).find('table').find_next('table').findAll('tr')
        inner_dictionary = {}

        for row in body:
            cols = row.findChildren(recursive=False)
            cols = [ele.text.strip() for ele in cols]
            inner_dictionary.update({cols[0] : cols[1]})
        return inner_dictionary
    except:
        pass

In [168]:
def get_factsheet(soup):
    try:
        factsheet = soup.find('a', {'class' : 'm50-product-detail-with-table-module__download__item__description'})
        factsheet = factsheet.get('href')
        return factsheet
    except:
        pass

In [126]:
def get_files(soup):
    try:
        titles_lst = []
        links_lst = []
        files_dic = {}

        titles = soup.findAll('a', {'class' : 'm14-accordion-module__navigation__toggler'})
        for t in titles:
            title = t.text
            titles_lst.append(title)

        files = soup.findAll('a', {'class' : 'm102-1-download-item-module__item'})
        for f in files:
            file = f.get('href')
            links_lst.append(file)

        for n in range(len(links_lst)):
            if len(links_lst) > len(titles_lst):
                titles_lst.append('RFA Files')
            files_dic.update({ titles_lst[n] : links_lst[n] })
            
        return files_dic
    
    except:
        pass

In [134]:
df['soup'] = df['webpage_to_be_scraped'].apply(lambda x: create_soup(x))

In [136]:
df['item name'] = df['soup'].apply(lambda x: item_name(x))

In [146]:
df['item description'] = df['soup'].apply(lambda x: item_description(x))

In [149]:
df['product_number_table'] = df['soup'].apply(lambda x: product_number_table(x))

In [153]:
df_temp = df['product_number_table'].apply(pd.Series)

In [155]:
df = pd.concat([df, df_temp], axis=1)

In [159]:
df['product_properties_table'] = df['soup'].apply(lambda x: product_properties_table(x))

In [161]:
df_temp = df['product_properties_table'].apply(pd.Series)

In [163]:
df = pd.concat([df, df_temp], axis=1)

In [171]:
df['Factsheet'] = df['soup'].apply(lambda x: get_factsheet(x))

In [173]:
df['files'] = df['soup'].apply(lambda x: get_files(x))

In [175]:
df_temp = df['files'].apply(pd.Series)

In [177]:
df = pd.concat([df, df_temp], axis=1)

In [179]:
df.to_excel('Franke_complete.xlsx')